# Определение стоимости автомобилей

Необходимо разработать приложение для привлечения новых клиентов для сервиса по продаже автомобилей с пробегом. Оно должно позволять быстро узнать рыночную стоимость автомобиля на основе исторических данных о технических характеристиках, комплектациях и ценах автомобилей. 

Для этого  необходимо построить модель, которая обеспечивает высокое качество предсказания, быстродействие и краткое время обучения. Заказчику важен точный прогноз стоимости, быстрый расчёт этой стоимости при использовании приложения и быстрое обучение модели.

## Подготовка данных

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
try:
    data = pd.read_csv('/Users/pashc/Downloads/autos.csv')
except:
    data = pd.read_csv('/datasets/autos.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [ ]:
data.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

В некоторых столбцах таблицы имеются пропущенные значения, которые не могут быть заменены на основании других столбцов. Если заменить их на значение, такое как "неизвестно", то это может привести к созданию выбросов. Однако, так как доля пропусков невелика по сравнению со всем объемом данных, можно удалить строки с пропущенными значениями.

In [ ]:
data["VehicleType"] = data["VehicleType"].fillna(data["VehicleType"].mode().values[0])
data["Gearbox"] = data["Gearbox"].fillna(data["Gearbox"].mode().values[0])
data["Model"] = data["Model"].fillna(data["Model"].mode().values[0])
data["FuelType"] = data["FuelType"].fillna(data["FuelType"].mode().values[0])
data["Repaired"] = data["Repaired"].fillna(data["Repaired"].mode().values[0])

Переведем в нижний регестр название столбцов.

In [ ]:
data.columns = data.columns.str.lower()

Посмотрим есть ли дубликаты, и в случае если таковые имеются удалим их.

In [ ]:
data.duplicated().sum()

5

In [ ]:
data = data.drop_duplicates().reset_index(drop=True)

Для упрощения работы с данными и исключения ненужной информации, мы удалим столбцы, которые не имеют важности для дальнейшего анализа.

In [ ]:
data = data.drop(["datecrawled","datecreated","lastseen","numberofpictures","postalcode"],axis = 1)

In [ ]:
data.head()

,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,repaired
0,480,sedan,1993,manual,0,golf,150000,0,petrol,volkswagen,no
1,18300,coupe,2011,manual,190,golf,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,no
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no


Проверим наличие аномалий.

In [ ]:
data.price.unique()

array([  480, 18300,  9800, ..., 12395, 18429, 10985])

In [ ]:
data.vehicletype.unique()

array(['sedan', 'coupe', 'suv', 'small', 'convertible', 'bus', 'wagon',
       'other'], dtype=object)

In [ ]:
data.registrationyear.unique()

array([1993, 2011, 2004, 2001, 2008, 1995, 1980, 2014, 1998, 2005, 1910,
       2016, 2007, 2009, 2002, 2018, 1997, 1990, 2017, 1981, 2003, 1994,
       1991, 1984, 2006, 1999, 2012, 2010, 2000, 1992, 2013, 1996, 1985,
       1989, 2015, 1982, 1976, 1983, 1973, 1111, 1969, 1971, 1987, 1986,
       1988, 1970, 1965, 1945, 1925, 1974, 1979, 1955, 1978, 1972, 1968,
       1977, 1961, 1960, 1966, 1975, 1963, 1964, 5000, 1954, 1958, 1967,
       1959, 9999, 1956, 3200, 1000, 1941, 8888, 1500, 2200, 4100, 1962,
       1929, 1957, 1940, 3000, 2066, 1949, 2019, 1937, 1951, 1800, 1953,
       1234, 8000, 5300, 9000, 2900, 6000, 5900, 5911, 1933, 1400, 1950,
       4000, 1948, 1952, 1200, 8500, 1932, 1255, 3700, 3800, 4800, 1942,
       7000, 1935, 1936, 6500, 1923, 2290, 2500, 1930, 1001, 9450, 1944,
       1943, 1934, 1938, 1688, 2800, 1253, 1928, 1919, 5555, 5600, 1600,
       2222, 1039, 9996, 1300, 8455, 1931, 1915, 4500, 1920, 1602, 7800,
       9229, 1947, 1927, 7100, 8200, 1946, 7500, 35

Были обнаружены аномалии в данных. Для исключения их влияния в анализе, мы исключим значения до 1900 года и после 2016 года из дальнейшей обработки.

In [ ]:
data = data.loc[(data['registrationyear'] <= 2016) & (data['registrationyear'] >= 1900)]

In [ ]:
data.gearbox.unique()

array(['manual', 'auto'], dtype=object)

In [ ]:
data.power.unique()

array([    0,   190,   163,    75,    69,   102,   109,    50,   125,
         101,   105,   140,   115,   131,    60,   136,   160,   231,
          90,   118,   193,    99,   113,   218,   122,   129,    70,
         306,    95,    61,   177,    80,   170,    55,   143,    64,
         286,   232,   150,   156,    82,   155,    54,   185,    87,
         180,    86,    84,   224,   235,   200,   178,   265,    77,
         110,   144,   120,   116,   184,   126,   204,    88,   194,
         305,   197,   179,   250,    45,   313,    41,   165,    98,
         130,   114,   211,    56,   201,   213,    58,   107,    83,
         174,   100,   220,    85,    73,   192,    68,    66,   299,
          74,    52,   147,    65,   310,    71,    97,   239,   203,
           5,   300,   103,   258,   320,    63,    81,   148,    44,
         145,   230,   280,   260,   104,   188,   333,   186,   117,
         141,    59,   132,   234,   158,    39,    92,    51,   135,
          53,   209,

Была обнаружена аномалия в данных. Для дальнейшей обработки, мы не будем включать значения силы выше 700.

In [ ]:
data = data.loc[data['power'] <= 700]

In [ ]:
data.model.unique()

array(['golf', 'grand', 'fabia', '3er', '2_reihe', 'other', 'c_max',
       '3_reihe', 'passat', 'navara', 'ka', 'polo', 'twingo', 'a_klasse',
       'scirocco', '5er', 'meriva', 'arosa', 'c4', 'civic', 'transporter',
       'punto', 'e_klasse', 'clio', 'kadett', 'kangoo', 'corsa', 'one',
       'fortwo', '1er', 'b_klasse', 'signum', 'astra', 'a8', 'jetta',
       'fiesta', 'c_klasse', 'micra', 'vito', 'sprinter', '156', 'escort',
       'forester', 'xc_reihe', 'scenic', 'a4', 'a1', 'insignia', 'combo',
       'focus', 'tt', 'a6', 'jazz', 'omega', 'slk', '7er', '80', '147',
       '100', 'z_reihe', 'sportage', 'sorento', 'v40', 'ibiza', 'mustang',
       'eos', 'touran', 'getz', 'a3', 'almera', 'megane', 'lupo', 'r19',
       'zafira', 'caddy', 'mondeo', 'cordoba', 'colt', 'impreza',
       'vectra', 'berlingo', 'tiguan', 'i_reihe', 'espace', 'sharan',
       '6_reihe', 'panda', 'up', 'seicento', 'ceed', '5_reihe', 'yeti',
       'octavia', 'mii', 'rx_reihe', '6er', 'modus', 'fox', 'ma

In [ ]:
data.kilometer.unique()

array([150000, 125000,  90000,  40000,  30000,  70000,   5000, 100000,
        60000,  20000,  80000,  50000,  10000])

In [ ]:
data.registrationmonth.unique()

array([ 0,  5,  8,  6,  7, 10, 12, 11,  2,  3,  1,  4,  9])

In [ ]:
data.fueltype.unique()

array(['petrol', 'gasoline', 'lpg', 'other', 'hybrid', 'cng', 'electric'],
      dtype=object)

In [ ]:
data.brand.unique()

array(['volkswagen', 'audi', 'jeep', 'skoda', 'bmw', 'peugeot', 'ford',
       'mazda', 'nissan', 'renault', 'mercedes_benz', 'opel', 'seat',
       'citroen', 'honda', 'fiat', 'mini', 'smart', 'hyundai',
       'sonstige_autos', 'alfa_romeo', 'subaru', 'volvo', 'mitsubishi',
       'kia', 'suzuki', 'lancia', 'toyota', 'chevrolet', 'dacia',
       'daihatsu', 'trabant', 'saab', 'chrysler', 'jaguar', 'daewoo',
       'porsche', 'rover', 'land_rover', 'lada'], dtype=object)

In [ ]:
data.repaired.unique()

array(['no', 'yes'], dtype=object)

In [ ]:
data.duplicated().sum()

33246

В первом разделе произведена обработка данных, которая включала удаление пропущенных значений и дубликатов. Также были убраны столбцы, которые не несут важной информации. Были проанализированы и обработаны аномалии в столбцах, содержащих год регистрации и лошадиные силы машин.

## Обучение моделей

In [ ]:
features=data.drop(['price'],axis=1)
target=data['price']

In [ ]:
col=['vehicletype', 'gearbox','model',
       'fueltype', 'brand', 'repaired']

In [ ]:

features_train, features_valid_test, target_train, target_valid_test  = train_test_split(features,target, test_size=0.4, random_state=254)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid_test, target_valid_test, test_size=0.5, random_state=12345)

features_train.shape, features_valid.shape, features_test.shape

((212310, 10), (70770, 10), (70770, 10))

In [ ]:
# Копируем данные для кодировки
features_train_oe = features_train.copy()
features_valid_oe = features_valid.copy()
features_test_oe = features_test.copy()

In [ ]:
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
encoder.fit(features_train[col])                                   
features_train_oe[col] = encoder.transform(features_train_oe[col]) 
features_valid_oe[col] = encoder.transform(features_valid_oe[col]) 
features_test_oe[col] = encoder.transform(features_test_oe[col])  
features_test_oe.head()                                            

,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,repaired
92409,4.0,2004,0.0,218,240.0,150000,6,2.0,2.0,0.0
213880,1.0,2011,1.0,160,224.0,30000,4,6.0,1.0,0.0
125566,4.0,2015,0.0,170,124.0,20000,7,6.0,38.0,1.0
29005,0.0,2010,1.0,140,36.0,150000,4,2.0,30.0,0.0
249122,7.0,1997,1.0,0,42.0,150000,11,6.0,24.0,1.0


Обучим модель решающего дерева

In [ ]:
for depth in range(3, 20, 3):
    model = DecisionTreeRegressor(random_state=12345, max_depth = depth)
    model.fit(features_train_oe, target_train)
    predictions = model.predict(features_valid_oe)
    rmse = mean_squared_error(target_valid, predictions)**0.5
    print('Глубина:', depth)
    print('RMSE для решающего дерева:', rmse)
    print('')

Глубина: 3
RMSE для решающего дерева: 3019.141268027153

Глубина: 6
RMSE для решающего дерева: 2434.841849746518

Глубина: 9
RMSE для решающего дерева: 2165.260230790978

Глубина: 12
RMSE для решающего дерева: 2071.0648975038393

Глубина: 15
RMSE для решающего дерева: 2084.724893486856

Глубина: 18
RMSE для решающего дерева: 2150.8700726285556



В результате анализа была выбрана оптимальная глубина дерева в качестве гиперпараметра модели. Лучший результат метрики RMSE был достигнут при глубине равной 12 и составил 2071.0648975038393

Обучим модель случайного леса

In [ ]:
for depth in range(1,5):
    for est in range(5,20,5):
        model = RandomForestRegressor(max_depth=depth, n_estimators=est, random_state=1234)
        model.fit(features_train_oe, target_train)
        predictions = model.predict(features_valid_oe)
        rmse = mean_squared_error(target_valid, predictions)**0.5
        print('Количество деревьев:', est, "Глубина",depth)
        print('RMSE для случайного леса:', rmse)
        print('')

Количество деревьев: 5 Глубина 1
RMSE для случайного леса: 3777.6283430007106

Количество деревьев: 10 Глубина 1
RMSE для случайного леса: 3777.625659426334

Количество деревьев: 15 Глубина 1
RMSE для случайного леса: 3777.617900078378

Количество деревьев: 5 Глубина 2
RMSE для случайного леса: 3240.4628020355567

Количество деревьев: 10 Глубина 2
RMSE для случайного леса: 3241.037047276033

Количество деревьев: 15 Глубина 2
RMSE для случайного леса: 3241.3335758122657

Количество деревьев: 5 Глубина 3
RMSE для случайного леса: 2925.8126065928964

Количество деревьев: 10 Глубина 3
RMSE для случайного леса: 2925.215041846241

Количество деревьев: 15 Глубина 3
RMSE для случайного леса: 2920.9731600106134

Количество деревьев: 5 Глубина 4
RMSE для случайного леса: 2637.155010942584

Количество деревьев: 10 Глубина 4
RMSE для случайного леса: 2634.662730710407

Количество деревьев: 15 Глубина 4
RMSE для случайного леса: 2630.1522326469635



При обучении модели были использованы гиперпараметры, среди которых оптимальными были глубина дерева и количество деревьев. Лучший результат метрики RSME был достигнут при глубине, равной 4, количестве деревьев, равном 15, и составил 2630.15223264696.

Для использования модели LightGBM, необходимо перевести категориальные признаки в тип "category".

Обучим модель LGBMRegressor

In [ ]:
for est in [100, 500, 1000]:
    for num in [25, 400, 15]:
        model = lgb.LGBMRegressor(n_estimators = est, num_leaves = num, random_state=12345)
        model.fit(features_train_oe, target_train)
        predictions= model.predict(features_valid_oe)
        rmse = mean_squared_error(target_valid, predictions)**0.5
        print('Количество деревьев:', est, num)
        print('RMSE для случайного леса:', rmse)
        print('')

Количество деревьев: 100 25
RMSE для случайного леса: 1880.196945697684

Количество деревьев: 100 400
RMSE для случайного леса: 1720.1503016139113

Количество деревьев: 100 15
RMSE для случайного леса: 1929.7644375587054

Количество деревьев: 500 25
RMSE для случайного леса: 1764.4245819772593

Количество деревьев: 500 400
RMSE для случайного леса: 1708.3691569638047

Количество деревьев: 500 15
RMSE для случайного леса: 1795.872183052806

Количество деревьев: 1000 25
RMSE для случайного леса: 1732.17974337289



В результате обучения трех моделей - решающего дерева, случайного леса и LightGBM, лучшим результатом по метрике RMSE с показателем в 1708.3691569638047 показала себя модель LightGBM.

## Анализ моделей

Для каждой модели будет рассчитано время обучения и время предсказания на тренировочной выборке.

In [ ]:
%%time
model1 = DecisionTreeRegressor(random_state=12345, max_depth=18)
model1.fit(features_train_oe, target_train)


In [ ]:
%%time
predictions = model1.predict(features_test_oe)
rmse = mean_squared_error(target_test, predictions)**0.5
print(rmse)

In [ ]:
%%time
model2 = RandomForestRegressor(random_state=12345, max_depth=4, n_estimators=15)
model2.fit(features_train_oe, target_train)

In [ ]:
%%time
predictions = model2.predict(features_test)
rmse = mean_squared_error(target_test_oe, predictions)**0.5
print(rmse)

In [ ]:
%%time
model3 = lgb.LGBMRegressor(random_state=12345,
                          n_estimators=1000,
                          num_leaves=400)
model3.fit(features_train_oe, target_train)

In [ ]:
%%time
predictions = model3.predict(features_test)
rmse = mean_squared_error(target_test_oe, predictions)**0.5
print(rmse)

По результату метрики RMSE, лучший результат показала модель решающего дерева. На втором месте расположилась модель LGBMRegressor, а на третьем - модель RandomForestRegressor.

## Финальны вывод

В процессе работы была проведена подготовка данных, в рамках которой были обнаружены аномалии, дубликаты и пропущенные значения. Аномалии и ненужные столбцы были исключены из рассмотрения, а дубликаты и пропущенные значения были удалены.

Были рассмотрены три модели - случайный лес, решающее дерево и LGBMRegressor, с подбором оптимальных параметров и оценкой качества моделей на метрике RMSE. Лучшей моделью с показателем RMSE 1708.3691569638047 является LGBMRegressor. 

При анализе времени обучения моделей на тренировочной выборке, модели расположились в следующем порядке: решающее дерево, LGBMRegressor, случайный лес.

Для заказчика можно порекомендовать использование модели LGBMRegressor, которая обладает оптимальными характеристиками, как на метрике RMSE, так и по скорости обучения и предсказания.